In [1]:
%matplotlib inline

import numpy as np
from matplotlib import pyplot as plt

%load_ext autoreload
%autoreload 2

In [2]:
import sys
import pathlib

root = pathlib.Path('../..')
sys.path.append(str(root))

In [3]:
s1product = root / '..' / 'data' /' S1B_IW_SLC__1SDV_20190805T162509_20190805T162536_017453_020D3A_BE8E.SAFE'

In [4]:
from lxml import etree
from scipy.constants import c

def read_s1_annotations(product_path):
    # NOTE: only reads annotations needed by this example
    product_path = pathlib.Path(product_path)
    metadata_root = product_path / 'annotation'
    metadata = {}
    for metadata_file in metadata_root.glob('s1[ab]-???-slc-[hv][hv]-*-???.xml'):
        xmldoc = etree.parse(str(metadata_file))
        nsmap = xmldoc.getroot().nsmap
        xp = 'imageAnnotation/imageInformation/azimuthTimeInterval'
        daz_s = float(xmldoc.findtext(xp, namespaces=nsmap))
        xp = 'imageAnnotation/imageInformation/rangePixelSpacing'
        drg_m = float(xmldoc.findtext(xp, namespaces=nsmap))
        drg_s = drg_m * 2 / c
        metadata[metadata_file.stem] = dict(drg_s=drg_s, daz_s=daz_s)
        
    return metadata

metadata = read_s1_annotations(s1product)
s1_metadata = list(metadata.values())[0]
s1_metadata

IndexError: list index out of range

In [ ]:
import s1etad

In [ ]:
path = root / 'test' / 'S1B_IW_ETA__AXDV_20190805T162509_20190805T162536_017453_020D3A_____.SAFE'
product = s1etad.Sentinel1Etad(path)
product

In [ ]:
swath = product['IW3']
swath

In [ ]:
burst = swath[3]
burst

In [ ]:
kxf = burst.sampling['x']/s1_metadata['drg_s']
kyf = burst.sampling['y']/s1_metadata['daz_s']
kx = round(kxf)
ky = round(kyf)
print('Grid scaling factors:')
print(f'  x: {kx} (err: {kx - kxf:.3g})')
print(f'  y: {ky} (err: {ky - kyf:.3g})')

In [ ]:
# read corrections
az, rg = burst.get_burst_grid()
correction_m = burst.get_sum_correction(meter=True)

In [ ]:
extent=[rg[0]*1e6, rg[-1]*1e6, az[0], az[-1]]
fig, ax = plt.subplots(1, 2, figsize=[13,5])

img = ax[0].imshow(correction_m['x'], aspect='auto', extent=extent)
fig.colorbar(img, ax=ax[0]).set_label('[m]')
ax[0].grid()
ax[0].set_xlabel(r'$\tau\ [\mu s]$')
ax[0].set_ylabel('t [s]')
ax[0].set_title('Range')

img = ax[1].imshow(correction_m['y'], aspect='auto', extent=extent)
fig.colorbar(img, ax=ax[1]).set_label('[m]')
ax[1].grid()
ax[1].set_xlabel(r'$\tau\ [\mu s]$')
ax[1].set_ylabel('t [s]')
ax[1].set_title('Azimuth')

fig.suptitle(f'Sum of corrections for {burst.swath_id} Burst{burst.burst_index:04d}')

In [ ]:
# resample to SLC sampling

from scipy.interpolate import RectBivariateSpline

az, rg = burst.get_burst_grid()
correction_s = burst.get_sum_correction()

# relatve coordinates
n_az_slc = (len(az) - 1) * ky + 1
n_rg_slc = (len(rg) - 1) * kx + 1

az_slc_rel = np.arange(n_az_slc) / ky
rg_slc_rel = np.arange(n_rg_slc) / kx

az_slc = az[0] + az_slc_rel * burst.sampling['y']
rg_slc = rg[0] + rg_slc_rel * burst.sampling['x']

interpx = RectBivariateSpline(np.arange(len(az)), np.arange(len(rg)), correction_s['x'])
slc_x_correction_s = interpx(az_slc_rel, rg_slc_rel)
interpy = RectBivariateSpline(np.arange(len(az)), np.arange(len(rg)), correction_s['y'])
slc_y_correction_s = interpy(az_slc_rel, rg_slc_rel)

slc_x_correction_pix = slc_x_correction_s / s1_metadata['drg_s']
slc_y_correction_pix = slc_y_correction_s / s1_metadata['daz_s']

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=[13,5])

img = ax[0].imshow(slc_x_correction_pix, aspect='auto')
fig.colorbar(img, ax=ax[0]).set_label('[samples]')
ax[0].grid()
ax[0].set_xlabel('Range [samples]$')
ax[0].set_ylabel('Azimuth [samples]')
ax[0].set_title('Range')

img = ax[1].imshow(slc_y_correction_pix, aspect='auto')
fig.colorbar(img, ax=ax[1]).set_label('[samples]')
ax[1].grid()
ax[0].set_xlabel('Range [samples]$')
ax[0].set_ylabel('Azimuth [samples]')
ax[1].set_title('Azimuth')

fig.suptitle(f'Sum of corrections for {burst.swath_id} Burst{burst.burst_index:04d}')

In [ ]:
xslice = slice(24995, 25005)
yslice = slice(0, 5)
fig = plt.figure(figsize=[10, 7])
Y, X = np.meshgrid(np.arange(yslice.start, yslice.stop), np.arange(xslice.start, xslice.stop))
C = np.hypot(slc_y_correction_pix[yslice, xslice], slc_x_correction_pix[yslice, xslice])
plt.scatter(Y, X)
plt.quiver(Y, X,
           slc_y_correction_pix[yslice, xslice], slc_x_correction_pix[yslice, xslice], C,
           scale_units='xy', scale=1, angles='xy')
plt.grid()
plt.colorbar()

In [ ]:
correction_m['x'].shape

In [ ]:
burst

In [ ]:
burst._grp

In [ ]:
iw1

In [ ]:
b1 = iw1[1]
b1._grp

In [ ]:
(2 - 1) * 14 + 1

In [ ]:
len(az), len(rg), az_slc_rel[-1], rg_slc_rel[-1]

In [ ]:
from scipy import ndimage

In [ ]:
ndimage.maximum_position(np.abs(slc_x_correction_pix))

In [ ]:
slc_x_correction_pix.shape

In [ ]:
slc_x_correction_pix[yslice, xslice].shape

In [ ]:
slc_x_correction_pix[yslice, xslice].max(), slc_y_correction_pix[yslice, xslice].max()

In [ ]:
np.arange(slice(3,8)